<a href="https://colab.research.google.com/github/Danddt/Russian_Opinion_Polls/blob/main/scrapping_NOVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 7.8 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16368 sha256=1e277e04ad187b1b5eee0813a09a25faef553f4a06c65e91803ea9a838165d48
  Stored in directory: /root/.cache/pip/wheels/ae/e1/6c/5137bc3f35aa130deea71575e165cc4f4f0680a88f3d90a636
Successfully built googletrans


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
urls = ['https://nova.bg/search/news?q=%D0%BF%D1%83%D1%82%D0%B8%D0%BD', 'https://nova.bg/search/news/2/?q=%D0%BF%D1%83%D1%82%D0%B8%D0%BD','https://nova.bg/search/news/3/?q=%D0%BF%D1%83%D1%82%D0%B8%D0%BD']

In [ ]:
def get_info(url):
  response= requests.get(url).content
  soup = BeautifulSoup(response, "html.parser")
  return soup

In [ ]:
list_articles = []

def list_art(soup):
  listo = soup.find_all('div',class_='media-cont')
  for title in listo:
    temp = (title.find('a').attrs['href'])
    list_articles.append(temp)
  return list_articles

In [ ]:
# Getting list of articles

all_art_link= []

for art in urls:
  temp = get_info(art)
  listo_temp = list_art(temp)
  all_art_link.append(listo_temp)



In [ ]:
flat_art_link = [element for innerList in all_art_link for element in innerList]

In [ ]:
# Extracting each article data

import re

regex = re.compile(r'<[^>]+>')


def get_data_name(soup1):
  name = soup1.h1.get_text()
  return name

def get_data_art(soup1):
  texts =str(soup1.find(id="description-wrapper"))
  text = regex.sub('', texts)
  text= text.replace('\xa0', ' ')
  text= text.replace('\n', ' ')
  return text

def get_data_date(soup1):
  datum = soup1.find('meta', itemprop='datePublished', content=True)['content'][:10]
  return datum


In [ ]:
bodies_list  = []
date = []
article_name = []

for item in flat_art_link:
  link_soup = get_info(item)
  name=get_data_name(link_soup)
  article_name.append(name)
  content =get_data_art(link_soup)
  bodies_list .append(content)
  date_time = get_data_date(link_soup)
  date.append(date_time)



In [ ]:
zipped = list(zip(date, article_name, bodies_list, flat_art_link))

df_art = pd.DataFrame(zipped, columns=['Date', 'Title', 'Content', 'link'])


In [ ]:
df_art.to_csv('/content/drive/MyDrive/news/df_list_aarticles.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np

df_bg =pd.read_csv('/content/drive/MyDrive/news/df_list_aarticles.csv')

In [ ]:
df_bg

,Date,Title,Content,link
0,2023-04-03,Задържаха 26-годишна за убийството на руски во...,Руската полиция задържа жена за убийството на...,https://nova.bg/news/view/2023/04/03/407128/%D...
1,2023-04-03,ВОТЪТ В ЧУЖБИНА: Как гласуваха българите зад г...,Вече са обработени 98.91% от протоколите от в...,https://nova.bg/news/view/2023/04/03/407109/%D...
2,2023-04-01,Руският военен министър Сергей Шойгу обеща да ...,Русият военен министър Сергей Шойгу обеща да ...,https://nova.bg/news/view/2023/04/01/406878/%D...
3,2023-04-01,Ескалация на дипломатическото напрежение между...,Дипломатическото напрежение между Москва и За...,https://nova.bg/news/view/2023/04/01/406869/%D...
4,2023-04-01,ЕКСКЛУЗИВНО: Кайлаш Сатяртхи – нобеловият лаур...,"Кайлаш Сатяртхи е индийски реформатор, посвет...",https://nova.bg/news/view/2023/04/01/406838/%D...
...,...,...,...,...
166,2023-03-21,Японският премиер пристигна на изненадваща виз...,Японският премиер Фумио Кишида пристигна на и...,https://nova.bg/news/view/2023/03/21/405446/%D...
167,2023-04-03,При 99.58% обработени протоколи: 6-партиен пар...,"Последните данни на ЦИК сочат, че при обработ...",https://nova.bg/news/view/2023/04/03/407101/%D...
168,2023-04-03,ПО ОБЛАСТИ: Как се разпредели вотът у нас,ЦИК публикува разбивката на резултите от пред...,https://nova.bg/news/view/2023/04/03/407112/%D...
169,2023-04-03,ОКОНЧАТЕЛНИ РЕЗУЛТАТИ: ГЕРБ-СДС печели вота с ...,"Шест партии влизат в 49-ото Народно събрание,...",https://nova.bg/news/view/2023/04/03/407135/%D...


In [ ]:
from googletrans import Translator
translator = Translator()

In [ ]:
df_bg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     171 non-null    object
 1   Title    171 non-null    object
 2   Content  171 non-null    object
 3   link     171 non-null    object
dtypes: object(4)
memory usage: 5.5+ KB


In [ ]:
df_bg['Title_en']=(df_bg['Title']).map(lambda x: translator.translate(x, src="bg", dest="en").text)
df_bg['Content_en']=(df_bg['Content']).map(lambda x: translator.translate(x, src="bg", dest="en").text)

In [ ]:
df_bg.to_csv('/content/drive/MyDrive/news/df_art_translated_junkin.csv', index=False)

In [ ]:
df_bg_first_clean = df_bg.drop_duplicates()

In [ ]:
df_bg_first_clean.to_csv('/content/drive/MyDrive/news/df_art_translated_junkin_first_clean.csv', index=False)

Manual translate due to character limimits

In [ ]:
with open('/content/drive/MyDrive/news/radev.txt', 'r') as file:
    radev = file.read().replace('\n', '')

In [ ]:
with open('/content/drive/MyDrive/news/Russia_has_nuclear_weapons_in_Belarus.txt', 'r') as file:
    belarus = file.read().replace('\n', '')

In [ ]:
with open('/content/drive/MyDrive/news/Petro_Poroshenko.txt', 'r') as file:
    poroshenko = file.read().replace('\n', '')

In [ ]:
df_manual_celan = pd.read_csv('/content/drive/MyDrive/news/df_art_translated_manual_clean.csv')

In [ ]:
df_manual_celan

In [ ]:
new_row_belarus = df_manual_celan.loc[21].to_list()[:-1]
new_row_belarus.append(belarus)
belaru_df = pd.DataFrame(new_row_belarus).T

In [ ]:
new_row_poroshenko = df_manual_celan.loc[5].to_list()[:-1]
new_row_poroshenko.append(poroshenko)
poroshenko_df = pd.DataFrame(new_row_poroshenko).T

In [ ]:
new_row_radev = df_manual_celan.loc[27].to_list()[:-1]
new_row_radev.append(radev)
radev_df = pd.DataFrame(new_row_radev).T

In [ ]:
to_be= [belaru_df,poroshenko_df, radev_df]

In [ ]:
to_be_df = pd.concat(to_be)
to_be_df.columns = df_manual_celan.columns.to_list()

In [ ]:
bg_clen_2 =df_manual_celan.drop([5, 21, 27])

In [ ]:
bg_final_clean =pd.concat([bg_clen_2, to_be_df]).reset_index()

In [ ]:
bg_final_clean.to_csv('/content/drive/MyDrive/news/bg_articles_clean_final.csv', index=False)

In [ ]:
# !pip install translate